# Install Python Kaggle Library

In [ ]:
! pip install -q kaggle

# Import Kaggle API KEY
Upload your kaggle.json file

In [ ]:
from google.colab import files
files.upload()

# Setup Kaggle Key Configuration

In [ ]:
 !mkdir ~/.kaggle
 !cp kaggle.json ~/.kaggle/
 !chmod 600 ~/.kaggle/kaggle.json

# Download Dataset

In [ ]:
!kaggle datasets download -d canggih/indonesian-food-recipes

  0% 0.00/3.63M [00:00<?, ?B/s]
100% 3.63M/3.63M [00:00<00:00, 88.3MB/s]


# Unzip Dataset

In [ ]:
!mkdir recipes
!unzip indonesian-food-recipes.zip -d recipes

Archive:  indonesian-food-recipes.zip
  inflating: recipes/dataset-ayam.csv  
  inflating: recipes/dataset-ikan.csv  
  inflating: recipes/dataset-kambing.csv  
  inflating: recipes/dataset-sapi.csv  
  inflating: recipes/dataset-tahu.csv  
  inflating: recipes/dataset-telur.csv  
  inflating: recipes/dataset-tempe.csv  
  inflating: recipes/dataset-udang.csv  


# Installing python package fake-useragent

In [ ]:
!pip install fake-useragent

In [ ]:
!pip install zenrows

# Initialize get_useragent function

In [ ]:
import random

def get_useragent():
  user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
  ]
  return random.choice(user_agent_list)

get_useragent()

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

# Initialize and test request_cookpad function

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from zenrows import ZenRowsClient
import time

def request_cookpad(path):
  attempts = 5
  sleep_attempt = 0.1
  image, description = None, None

  for trial in range(attempts):
    try:
      url = "https://cookpad.com" + path
      client = ZenRowsClient("")
      proxy = ""
      proxies = {"http": proxy, "https": proxy}
      response = requests.get(url, proxies=proxies, verify=False)
      html = response.text
      soup = BeautifulSoup(html, 'html.parser')

      img = soup.select('#recipe_image img')
      meta = soup.select('meta[property="twitter:description"]')
      image = img[0].get('src')
      description = meta[0].get('content')
    except Exception as error:
      time.sleep(sleep_attempt)

    if (not image is None and not description is None):
      break

  return image, description

image, description = request_cookpad('/id/resep/4448366-lumpia-udang-kulit-tahu-ala-fe')
print(image)
print(description)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.zenrows.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://img-global.cpcdn.com/recipes/78575c69e275a5ab/680x482cq70/lumpia-udang-kulit-tahu-ala-fe-foto-resep-utama.jpg
Ini menu dimsum favorit Mion klo ke resto Eastern IP. Jadi mommy berusaha keras untuk bisa bikin sendiri, biar puas makannya, karena dia bisa habis 5 porsi sekali makan 😅😅
Bisa awet sebulan disimpan difreezer karena sudah dikukus matang sempurna, walau jarang bertahan lebih dari 2 hari karena udah habis dimakan dia sendiri 🤣
Saya tambah sedikit ayam fillet buat ganjal aja biar ga terlalu udang banget 😁 jadi klo mau full udang silahkan disesuaikan saja.


# Initialize scrap_imagedesc and exec_time functions

In [ ]:
import time

# Scraping data
def scrap_imagedesc(dataframe, func, limit, cooldown_time=0):
  df = dataframe
  cooldown = 1
  cooldown_count = 1
  if limit is None:
    len(df)

  for index, row in df.iterrows():
      if (index < limit):
        path = row["URL"]
        if df.at[index, "image"] is None or df.at[index, "description"] is None:
          image, description = func(path)

          if df.at[index, "image"] is None:
            df.at[index, "image"] = image
          if df.at[index, "description"] is None:
            if not image is None and description is None:
              df.at[index, "description"] = ""
            else:
              df.at[index, "description"] = description
        if cooldown_count == cooldown:
          time.sleep(cooldown_time)
          cooldown_count = 1
        else:
          cooldown_count += 1
      else:
        break
  return df

def exec_time(func, args):
  start_time = time.time()
  func_return = func(*args)
  return time.time() - start_time, func_return

# Load one dataset and scrap 5 image and description using request_cookpad function





In [ ]:
import pandas as pd
file_name = "dataset-udang.csv"
df = pd.read_csv("recipes/"+file_name)

# Add description and image columns
df = df.assign(description = None)
df = df.assign(image = None)

seconds, df = exec_time(func=scrap_imagedesc, args=(df, request_cookpad, 50, 4))
print("Scraping time is %s seconds" % seconds)
df.head()

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.zenrows.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.zenrows.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.zenrows.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpoo

Scraping time is 443.7820179462433 seconds


,Title,Ingredients,Steps,Loves,URL,description,image
0,Lumpia udang kulit tahu ala fe',50 gram ayam potong kotak kecil--200 gram udan...,Campur ayam & udang dengan semua bumbu & daun ...,4,/id/resep/4448366-lumpia-udang-kulit-tahu-ala-fe,Ini menu dimsum favorit Mion klo ke resto East...,https://img-global.cpcdn.com/recipes/78575c69e...
1,Bakso Ayam Udang Keto,400 gr ayam giling--250 gr udang kupas--3 telu...,"Campur semua bahan dalam food processor, kecua...",2,/id/resep/4474544-bakso-ayam-udang-keto,"Tanpa tepung, buat yg rindu empek2 atau otak2 ...",https://img-global.cpcdn.com/recipes/5e3b6b0da...
2,Udang ala pop corn,1/4 kg udang basah ukuran sedang--1 bungkus ko...,Buang kepala dan cangkang udang.--Cuci bersih ...,2,/id/resep/4418156-udang-ala-pop-corn,"Suami masuk kerja pagi2 bgt, alhasil masak sea...",https://img-global.cpcdn.com/recipes/2589bc69d...
3,Bakwan Udang Sederhana,1/2 kg Tepung terigu (sudah di ayak)--1/2 Udan...,"Haluskan bawang putih, garam, picin dan merica...",0,/id/resep/4475500-bakwan-udang-sederhana,Kefikiran aja ni mom's hujan hujan berhubung p...,https://img-global.cpcdn.com/recipes/6b4f76a89...
4,Udang Goreng Tepung Mayonaise,300 gr udang (buang kepalanya)--Jeruk nipis--T...,"Cuci bersih udang, tambahkan jeruk nipis supay...",6,/id/resep/4474156-udang-goreng-tepung-mayonaise,Simpel 😊\nGampang bikin nya \nRequest anak ter...,https://img-global.cpcdn.com/recipes/f1aba92ff...


In [ ]:
df[0:50]

,Title,Ingredients,Steps,Loves,URL,description,image
0,Lumpia udang kulit tahu ala fe',50 gram ayam potong kotak kecil--200 gram udan...,Campur ayam & udang dengan semua bumbu & daun ...,4,/id/resep/4448366-lumpia-udang-kulit-tahu-ala-fe,Ini menu dimsum favorit Mion klo ke resto East...,https://img-global.cpcdn.com/recipes/78575c69e...
1,Bakso Ayam Udang Keto,400 gr ayam giling--250 gr udang kupas--3 telu...,"Campur semua bahan dalam food processor, kecua...",2,/id/resep/4474544-bakso-ayam-udang-keto,"Tanpa tepung, buat yg rindu empek2 atau otak2 ...",https://img-global.cpcdn.com/recipes/5e3b6b0da...
2,Udang ala pop corn,1/4 kg udang basah ukuran sedang--1 bungkus ko...,Buang kepala dan cangkang udang.--Cuci bersih ...,2,/id/resep/4418156-udang-ala-pop-corn,"Suami masuk kerja pagi2 bgt, alhasil masak sea...",https://img-global.cpcdn.com/recipes/2589bc69d...
3,Bakwan Udang Sederhana,1/2 kg Tepung terigu (sudah di ayak)--1/2 Udan...,"Haluskan bawang putih, garam, picin dan merica...",0,/id/resep/4475500-bakwan-udang-sederhana,Kefikiran aja ni mom's hujan hujan berhubung p...,https://img-global.cpcdn.com/recipes/6b4f76a89...
4,Udang Goreng Tepung Mayonaise,300 gr udang (buang kepalanya)--Jeruk nipis--T...,"Cuci bersih udang, tambahkan jeruk nipis supay...",6,/id/resep/4474156-udang-goreng-tepung-mayonaise,Simpel 😊\nGampang bikin nya \nRequest anak ter...,https://img-global.cpcdn.com/recipes/f1aba92ff...
5,Udang saus asam manis,1 Kg Udang mutiara--7 siung bawang merah--4 si...,"Iris bawang merah, bawang putih, jahe, lengkua...",0,/id/resep/4443123-udang-saus-asam-manis,,https://img-global.cpcdn.com/recipes/733e8d6d9...
6,Tekwan Kuah Udang,Untuk Tekwan :--1/4 ekor ikan tenggiri 🌸--2 bu...,"Untuk membuat tekwan, rebus air dengan 2sdm mi...",5,/id/resep/4464484-tekwan-kuah-udang,Bukan ‘wong kito galo’ tapi selalu suka sama m...,https://img-global.cpcdn.com/recipes/b47d5835a...
7,Sambal udang goreng,1/4 kg udang--2 buah jeruk nipis--Minyak goren...,"Cuci udang bersih, lumuri dengan jeruk nipis--...",7,/id/resep/4474006-sambal-udang-goreng,,https://img-global.cpcdn.com/recipes/838d97d50...
8,Udang asam manis oregano,1/2 kg udang vaname--1 bh bawang bombai--2 siu...,"Bersihkan udang dan cuci dengan air garam, iri...",3,/id/resep/4473599-udang-asam-manis-oregano,,https://img-global.cpcdn.com/recipes/eee84de83...
9,Tumis buncis pakai udang,1/4 kg Buncis (iris serong)--4 bh bawang putih...,"Tumis bawang putih sampai harum, masukan udang...",5,/id/resep/4473387-tumis-buncis-pakai-udang,Berbagi resep yg mudah dan simple tp yummy,https://img-global.cpcdn.com/recipes/c00c95870...


# Export dataset with new image and description columns data

In [ ]:
import os

!mkdir "prepared_recipes"
df = df[0:50]

file_path = "prepared_recipes/"+file_name
if os.path.exists(file_path):
  os.remove(file_path)
df.to_csv(file_path)

mkdir: cannot create directory ‘prepared_recipes’: File exists
